## Imports

In [142]:
import sqlite3
import pandas as pd
import sqlalchemy as sq


#### Conect to sqlite

In [143]:
engine =sq.create_engine("sqlite:///DBs/cademycode.db")
table_lst = sq.inspect(engine).get_table_names()
print(table_lst)
# we see only 3 tables in the DB let's look at the head and dtypes

['cademycode_courses', 'cademycode_student_jobs', 'cademycode_students']


In [144]:
# lets create a datafram dictionary of all the tables and show the heads
df_zip=zip(table_lst,[pd.read_sql_table(table, engine) for table in table_lst])
df_dic={table:df for (table,df) in df_zip}
for (table, df) in df_dic.items():
    print(table)
    display(df.describe() ,df.dtypes, df)


cademycode_courses


,career_path_id,hours_to_complete
count,10.00000,10.000000
mean,5.50000,21.900000
std,3.02765,6.707376
min,1.00000,12.000000
25%,3.25000,18.500000
50%,5.50000,20.000000
75%,7.75000,26.500000
max,10.00000,35.000000


career_path_id        int64
career_path_name     object
hours_to_complete     int64
dtype: object

,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18
5,6,frontend engineer,20
6,7,iOS developer,27
7,8,android developer,27
8,9,machine learning engineer,35
9,10,ux/ui designer,15


cademycode_student_jobs


...

,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000
5,6,education,61000
6,7,HR,80000
7,8,student,10000
8,9,healthcare,120000
9,0,other,80000


cademycode_students


,uuid
count,5000.000000
mean,2500.500000
std,1443.520003
min,1.000000
25%,1250.750000
50%,2500.500000
75%,3750.250000
max,5000.000000


uuid                       int64
name                      object
dob                       object
sex                       object
contact_info              object
job_id                    object
num_course_taken          object
current_career_path_id    object
time_spent_hrs            object
dtype: object

,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{'mailing_address': '303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{'mailing_address': '767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{'mailing_address': 'P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{'mailing_address': '517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{'mailing_address': '18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64
...,...,...,...,...,...,...,...,...,...
4995,4996,Quentin van Harn,1967-07-07,N,"{'mailing_address': '591 Blue Berry, Coulee, I...",5.0,5.0,2.0,13.82
4996,4997,Alejandro van der Sluijs,1964-11-03,M,"{'mailing_address': '30 Iron Divide, Pewaukee ...",4.0,13.0,1.0,7.86
4997,4998,Brock Mckenzie,2004-11-25,M,"{'mailing_address': '684 Rustic Rest Avenue, C...",8.0,10.0,3.0,12.1
4998,4999,Donnetta Dillard,1943-02-12,N,"{'mailing_address': '900 Indian Oval, Euclid, ...",3.0,6.0,5.0,14.86


#### Lets evaluate table by table
1. cademycode_courses look up table for career path no real issues here
2. cademycode_student_jobs look up table for student jobs no reall issues here
3. cademycode_students 
    - everything came in as a string so we have bad data types 
    - contact info contains mailing address and email thats not good in fact it should be its own table
    - mailing address should be broke up to street city state zip
    - name should be broke up between first and last name

So for this project the instructions state to make one big flat csv ill make a parquet as well

In [145]:
# check for missing data in students table
df_dic['cademycode_students'].count()
# we see missing values on job_id we could add an entry as "None" and replace them with that  and average salary
# we see missing values for number of cources taken we will replace that with 0
# we see missing values on current carrere_path_id we could add an entry as undecided and replace them with that
# the Time spent hours missing values is the same as the current carrer path this is structurally null so it will be replaced with 0

uuid                      5000
name                      5000
dob                       5000
sex                       5000
contact_info              5000
job_id                    4995
num_course_taken          4749
current_career_path_id    4529
time_spent_hrs            4529
dtype: int64

## Lets Get Cleaning, Spliting, and Merging

In [146]:
# fix data types missing values 
mdf=df_dic["cademycode_students"]
mdf["job_id"]=pd.to_numeric(mdf['job_id'].fillna(-1),downcast="integer")
mdf["num_course_taken"]=pd.to_numeric(mdf["num_course_taken"].fillna(0),downcast="integer")
mdf["current_career_path_id"]=pd.to_numeric(mdf["current_career_path_id"].fillna(0),downcast="integer")
mdf["time_spent_hrs"]=pd.to_numeric(mdf["time_spent_hrs"].fillna(0))
mdf["dob"]=pd.to_datetime(mdf["dob"])
mdf["first_name"]=mdf["name"].str.split(" ",expand=True)[0]
mdf["last_name"]=mdf["name"].str.split(" ",expand=True)[1]

def contact(ct_info:dict):
    maddress = ct_info["mailing_address"].split(",")
    eaddress = ct_info["email"]
    return pd.Series([*maddress, eaddress], index=["street", "city", "state", "zip", "email"])

mdf=pd.concat([mdf,mdf["contact_info"].apply(contact)],axis=1)
mdf.drop(columns=["contact_info", "name"], inplace=True)

# add the no jobs and unknown career paths
df_dic["cademycode_student_jobs"].loc[-1]=[-1, "none", 80000]
df_dic["cademycode_courses"].loc[-1]=[0, "undecided", 20]
big_df= mdf.merge(df_dic["cademycode_courses"], left_on="current_career_path_id", right_on="career_path_id")
big_df=big_df.merge(df_dic["cademycode_student_jobs"], on="job_id")

big_df.drop(columns=["job_id", "career_path_id"], inplace=True)

In [147]:
big_df


,uuid,dob,sex,num_course_taken,current_career_path_id,time_spent_hrs,first_name,last_name,street,city,state,zip,email,career_path_name,hours_to_complete,job_category,avg_salary
0,1,1943-07-03,F,6,1,4.99,Annabelle,Avery,303 N Timber Key,Irondale,Wisconsin,84736,annabelle_avery9376@woohoo.com,data scientist,20,HR,80000
1,2,1991-02-07,M,5,8,4.40,Micah,Rubio,767 Crescent Fair,Shoals,Indiana,37439,rubio6772@hmail.com,android developer,27,HR,80000
2,3,1989-12-07,M,8,8,6.74,Hosea,Dale,P.O. Box 41269,St. Bonaventure,Virginia,83637,hosea_dale8084@coldmail.com,android developer,27,HR,80000
3,4,1988-07-31,F,7,9,12.31,Mariann,Kirk,517 SE Wintergreen Isle,Lane,Arkansas,82242,kirk4005@hmail.com,machine learning engineer,35,education,61000
4,5,1963-08-31,M,14,3,5.64,Lucio,Alexander,18 Cinder Cliff,Doyles borough,Rhode Island,73737,alexander9810@hmail.com,data analyst,12,HR,80000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7001,4997,1964-11-03,M,13,1,7.86,Alejandro,van,30 Iron Divide,Pewaukee village,California,62992,alejandro4080@coldmail.com,data scientist,20,creative,66000
7002,4998,2004-11-25,M,10,3,12.10,Brock,Mckenzie,684 Rustic Rest Avenue,Carmine,California,55657,brock_mckenzie2025@inlook.com,data analyst,12,student,10000
7003,4999,1943-02-12,N,6,5,14.86,Donnetta,Dillard,900 Indian Oval,Euclid,Iowa,59683,dillard7526@inlook.com,backend engineer,18,software developer,110000
7004,4999,1943-02-12,N,6,5,14.86,Donnetta,Dillard,900 Indian Oval,Euclid,Iowa,59683,dillard7526@inlook.com,backend engineer,18,software developer,110000


In [135]:
ndf

,street,city,state,zip,email,uuid,dob,sex,job_id,num_course_taken,current_career_path_id,time_spent_hrs,first_name,last_name
0,303 N Timber Key,Irondale,Wisconsin,84736,annabelle_avery9376@woohoo.com,1,1943-07-03,F,7,6,1,4.99,Annabelle,Avery
1,767 Crescent Fair,Shoals,Indiana,37439,rubio6772@hmail.com,2,1991-02-07,M,7,5,8,4.40,Micah,Rubio
2,P.O. Box 41269,St. Bonaventure,Virginia,83637,hosea_dale8084@coldmail.com,3,1989-12-07,M,7,8,8,6.74,Hosea,Dale
3,517 SE Wintergreen Isle,Lane,Arkansas,82242,kirk4005@hmail.com,4,1988-07-31,F,6,7,9,12.31,Mariann,Kirk
4,18 Cinder Cliff,Doyles borough,Rhode Island,73737,alexander9810@hmail.com,5,1963-08-31,M,7,14,3,5.64,Lucio,Alexander
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,591 Blue Berry,Coulee,Illinois,65199,vanharn2778@woohoo.com,4996,1967-07-07,N,5,5,2,13.82,Quentin,van
4996,30 Iron Divide,Pewaukee village,California,62992,alejandro4080@coldmail.com,4997,1964-11-03,M,4,13,1,7.86,Alejandro,van
4997,684 Rustic Rest Avenue,Carmine,California,55657,brock_mckenzie2025@inlook.com,4998,2004-11-25,M,8,10,3,12.10,Brock,Mckenzie
4998,900 Indian Oval,Euclid,Iowa,59683,dillard7526@inlook.com,4999,1943-02-12,N,3,6,5,14.86,Donnetta,Dillard
